# Experimentos

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# time
import time
from datetime import timedelta

# metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ETL
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

# Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.backend import clear_session
from imblearn.over_sampling import SMOTE
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

# K-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

models = pd.DataFrame()

Using TensorFlow backend.


# Data

In [2]:
## Data
df = pd.read_csv('data/train.csv')

## Remove colunas constantes
colsToRemove = []
for col in df.columns:
    if df[col].std() == 0:
        colsToRemove.append(col)
df.drop(colsToRemove, axis=1, inplace=True)

## Remove colunas multiplas
colsToRemove = []
columns = df.columns
for i in range(len(columns)-1):
    print(i, end="\r")
    v = df[columns[i]]
    for j in range(i+1,len(columns)):
        m = df.loc[(df[columns[j]]*v) != 0, [columns[j], columns[i]]]
        if m.shape[0] > 0:
            c = m.iloc[0,0]/m.iloc[0,1]
            if np.array_equal(c*v.values,df[columns[j]].values):
                colsToRemove.append(columns[j])
df.drop(colsToRemove, axis=1, inplace=True)

## Remove duplicate rows
df.drop_duplicates(inplace=True)

## Split (X,Y) 
x_data = df.drop(['ID','TARGET'], axis=1)
y_data = df['TARGET'].copy()

x_data.shape

(76020, 287)

# SVM

In [3]:
# info
model_name = 'SVM(RBF)'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i+'            ', end="\r")
    
    ## Train
    startTime = time.time()
    model = SVC(C=0.1, kernel='rbf', class_weight = 'balanced')
    model.fit(x_train, y_train)
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred = model.predict(x_test)

    ## Metrics
    auc_train = roc_auc_score(y_train, model.predict(x_train))
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred)
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    i=i+1
    
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()
    
metrics_kf

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.726399,0.700542,0.700542,0.534536,0.700542,0.481041,0.687886,00:24:14.611310
KF_2,0.722076,0.721624,0.721624,0.537637,0.721624,0.482152,0.682494,00:23:03.004579
KF_3,0.725755,0.702319,0.702319,0.534851,0.702319,0.481539,0.688240,00:23:26.042186
KF_4,0.726799,0.698294,0.698294,0.534277,0.698294,0.481506,0.689687,00:23:30.962955
KF_5,0.726463,0.706703,0.706703,0.535344,0.706703,0.480484,0.684425,00:23:39.778466
KF_6,0.723543,0.713091,0.713091,0.536204,0.713091,0.480143,0.681400,00:23:24.295193
KF_7,0.725067,0.710454,0.710454,0.535838,0.710454,0.480192,0.682452,00:23:24.383880
KF_8,0.726333,0.671681,0.671681,0.530099,0.671681,0.478446,0.693633,00:23:28.400867
KF_9,0.725907,0.697442,0.697442,0.533648,0.697442,0.477870,0.682805,00:23:43.480291
KF_10,0.722965,0.699383,0.699383,0.533999,0.699383,0.478528,0.683463,00:23:30.173608


# Random Forest

In [4]:
# info
model_name = 'RF'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i+'            ', end="\r")
    
    ## Train
    startTime = time.time()
    model = RandomForestClassifier(
                                   n_estimators = 800,
                                   min_samples_leaf = 85,
                                   max_features = 0.3,
                                   class_weight = 'balanced',
                                   n_jobs=-1
                                  )
    model.fit(x_train, y_train)
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred = model.predict(x_test)
    y_pred_proba = model.predict_proba(x_test)[:,1]

    ## Metrics
    auc_train = roc_auc_score(y_train, model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba)
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    i=i+1

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()
    
metrics_kf

KF_10            


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.891783,0.832640,0.832640,0.558305,0.744186,0.558292,0.814547,00:03:07.567300
KF_2,0.888700,0.864993,0.864993,0.565929,0.790477,0.566144,0.808628,00:03:02.246706
KF_3,0.890878,0.838726,0.842048,0.562639,0.760630,0.565295,0.818600,00:03:02.167665
KF_4,0.891146,0.838793,0.838793,0.564327,0.768730,0.567590,0.818863,00:03:05.577534
KF_5,0.891183,0.816694,0.820016,0.554370,0.728264,0.552147,0.811497,00:03:03.427888
KF_6,0.890470,0.840036,0.843358,0.559838,0.765290,0.556377,0.803078,00:03:01.279845
KF_7,0.890959,0.834421,0.834421,0.561031,0.762755,0.560469,0.810445,00:03:03.102794
KF_8,0.890999,0.823296,0.823296,0.558404,0.746537,0.557895,0.812944,00:03:01.128614
KF_9,0.891617,0.827172,0.830505,0.559273,0.754419,0.558222,0.810683,00:03:01.300135
KF_10,0.891046,0.840121,0.840121,0.562272,0.769077,0.562291,0.811209,00:03:01.209207


# XGBoost

In [5]:
# info
model_name = 'XGBoost'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i+'            ', end="\r")
    
    ## Train
    startTime = time.time()
    model = XGBClassifier(
                          learning_rate = 0.03,
                          n_estimators = 350, 
                          objective = 'reg:logistic',
                          max_depth = 5,
                          subsample = 0.75,
                          colsample_bytree=0.7
                        )
    model.fit(x_train, y_train)
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred = model.predict(x_test)
    y_pred_proba = model.predict_proba(x_test)[:,1]

    ## Metrics
    auc_train = roc_auc_score(y_train, model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba)
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    i=i+1

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()
    
metrics_kf

KF_10            


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.890837,0.832903,0.836225,0.680324,0.503117,0.496402,0.960279,00:02:01.234849
KF_2,0.885953,0.866257,0.869579,0.980332,0.503322,0.496569,0.960673,00:02:01.870318
KF_3,0.889282,0.843243,0.843243,0.480195,0.499795,0.489799,0.960011,00:02:01.766925
KF_4,0.888785,0.842569,0.842569,0.480200,0.499932,0.489867,0.960274,00:02:01.132004
KF_5,0.889708,0.825224,0.825224,0.480200,0.499932,0.489867,0.960274,00:02:02.272340
KF_6,0.890410,0.845171,0.845171,0.480200,0.499932,0.489867,0.960274,00:02:03.566950
KF_7,0.889260,0.835608,0.835608,0.730263,0.501593,0.493200,0.960405,00:02:01.467451
KF_8,0.889576,0.828226,0.828226,0.730263,0.501593,0.493200,0.960405,00:02:01.917071
KF_9,0.889679,0.828749,0.832082,0.730326,0.501598,0.493244,0.960532,00:02:01.472406
KF_10,0.888830,0.847726,0.847726,0.813723,0.503265,0.496566,0.960663,00:02:01.331720


# MLP

In [3]:
# info
model_name = 'MLP'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Dense(500, activation='tanh', input_dim=x_train.shape[1]))
    model.add(Dense(80, activation='tanh'))
    model.add(Dropout(0.5)) 
    model.add(Dense(2, activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x_train,y_train,
                  batch_size=150,
                  epochs=10000,
                  callbacks=[EarlyStopping(patience=20)],
                  validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97447 samples, validate on 13684 samples
Epoch 1/10000
97447/97447 [==============================] - 13s - loss: 0.4624 - acc: 0.7970 - val_loss: 0.4315 - val_acc: 0.8055
Epoch 2/10000
97447/97447 [==============================] - 6s - loss: 0.3977 - acc: 0.8367 - val_loss: 0.3523 - val_acc: 0.8556
Epoch 3/10000
97447/97447 [==============================] - 6s - loss: 0.3610 - acc: 0.8527 - val_loss: 0.3868 - val_acc: 0.8470
Epoch 4/10000
97447/97447 [==============================] - 6s - loss: 0.3360 - acc: 0.8631 - val_loss: 0.3596 - val_acc: 0.8482
Epoch 5/10000
97447/97447 [==============================] - 6s - loss: 0.3206 - acc: 0.8700 - val_loss: 0.3660 - val_acc: 0.8440
Epoch 6/10000
97447/97447 [==============================] - 6s - loss: 0.3098 - acc: 0.8740 - val_loss: 0.3379 - val_acc: 0.8641
Epoch 7/10000
97447/97447 [==============================] - 6s - loss: 0.3002 - acc: 0.8789 - val_loss: 0.3532 - val_acc: 0.8574
Epoch 8/10000
97447/97447 [=======

97548/97548 [==============================] - 5s - loss: 0.2637 - acc: 0.8921 - val_loss: 0.2359 - val_acc: 0.8940
Epoch 18/10000
97548/97548 [==============================] - 5s - loss: 0.2609 - acc: 0.8949 - val_loss: 0.3366 - val_acc: 0.8568
Epoch 19/10000
97548/97548 [==============================] - 5s - loss: 0.2589 - acc: 0.8954 - val_loss: 0.2616 - val_acc: 0.8827
Epoch 20/10000
97548/97548 [==============================] - 5s - loss: 0.2564 - acc: 0.8961 - val_loss: 0.2980 - val_acc: 0.8710
Epoch 21/10000
97548/97548 [==============================] - 5s - loss: 0.2545 - acc: 0.8980 - val_loss: 0.3911 - val_acc: 0.8434
Epoch 22/10000
97548/97548 [==============================] - 5s - loss: 0.2514 - acc: 0.8986 - val_loss: 0.3457 - val_acc: 0.8579
Epoch 23/10000
97548/97548 [==============================] - 5s - loss: 0.2500 - acc: 0.8997 - val_loss: 0.2981 - val_acc: 0.8738
Epoch 24/10000
97548/97548 [==============================] - 5s - loss: 0.2482 - acc: 0.9002 - va

97662/97662 [==============================] - 5s - loss: 0.2261 - acc: 0.9112 - val_loss: 0.2613 - val_acc: 0.9047
Epoch 41/10000
97662/97662 [==============================] - 5s - loss: 0.2262 - acc: 0.9111 - val_loss: 0.2738 - val_acc: 0.8883
Epoch 42/10000
97312/97662 [============================>.] - ETA: 0sKF_4
Train on 97478 samples, validate on 13684 samples
Epoch 1/10000
97478/97478 [==============================] - 6s - loss: 0.4612 - acc: 0.7949 - val_loss: 0.4937 - val_acc: 0.7657
Epoch 2/10000
97478/97478 [==============================] - 5s - loss: 0.3939 - acc: 0.8375 - val_loss: 0.3789 - val_acc: 0.8467
Epoch 3/10000
97478/97478 [==============================] - 5s - loss: 0.3601 - acc: 0.8524 - val_loss: 0.3655 - val_acc: 0.8506
Epoch 4/10000
97478/97478 [==============================] - 5s - loss: 0.3405 - acc: 0.8606 - val_loss: 0.4060 - val_acc: 0.8346
Epoch 5/10000
97478/97478 [==============================] - 5s - loss: 0.3232 - acc: 0.8672 - val_loss: 0.32

97502/97502 [==============================] - 5s - loss: 0.3673 - acc: 0.8465 - val_loss: 0.4560 - val_acc: 0.8042
Epoch 4/10000
97502/97502 [==============================] - 6s - loss: 0.3481 - acc: 0.8543 - val_loss: 0.3781 - val_acc: 0.8270
Epoch 5/10000
97502/97502 [==============================] - 5s - loss: 0.3318 - acc: 0.8601 - val_loss: 0.3868 - val_acc: 0.8471
Epoch 6/10000
97502/97502 [==============================] - 5s - loss: 0.3203 - acc: 0.8647 - val_loss: 0.3567 - val_acc: 0.8489
Epoch 7/10000
97502/97502 [==============================] - 5s - loss: 0.3102 - acc: 0.8690 - val_loss: 0.2912 - val_acc: 0.8867
Epoch 8/10000
97502/97502 [==============================] - 6s - loss: 0.3049 - acc: 0.8733 - val_loss: 0.7168 - val_acc: 0.7511
Epoch 9/10000
97502/97502 [==============================] - 5s - loss: 0.3000 - acc: 0.8752 - val_loss: 0.3375 - val_acc: 0.8661
Epoch 10/10000
97502/97502 [==============================] - 5s - loss: 0.2913 - acc: 0.8796 - val_loss

97427/97427 [==============================] - 5s - loss: 0.2472 - acc: 0.9019 - val_loss: 0.2777 - val_acc: 0.8757
Epoch 22/10000
97427/97427 [==============================] - 5s - loss: 0.2453 - acc: 0.9026 - val_loss: 0.3219 - val_acc: 0.8677
Epoch 23/10000
97427/97427 [==============================] - 5s - loss: 0.2427 - acc: 0.9046 - val_loss: 0.3514 - val_acc: 0.8574
Epoch 24/10000
97427/97427 [==============================] - 6s - loss: 0.2409 - acc: 0.9056 - val_loss: 0.3874 - val_acc: 0.8564
Epoch 25/10000
97427/97427 [==============================] - 5s - loss: 0.2399 - acc: 0.9052 - val_loss: 0.3462 - val_acc: 0.8563
Epoch 26/10000
97427/97427 [==============================] - 5s - loss: 0.2376 - acc: 0.9070 - val_loss: 0.3122 - val_acc: 0.8656
Epoch 27/10000
97427/97427 [==============================] - 6s - loss: 0.2365 - acc: 0.9079 - val_loss: 0.3298 - val_acc: 0.8635
Epoch 28/10000
97427/97427 [==============================] - 6s - loss: 0.2337 - acc: 0.9091 - va

97561/97561 [==============================] - 5s - loss: 0.2113 - acc: 0.9178 - val_loss: 0.3602 - val_acc: 0.8568
Epoch 49/10000
97561/97561 [==============================] - 6s     
KF_8
Train on 97693 samples, validate on 13684 samples
Epoch 1/10000
97693/97693 [==============================] - 6s - loss: 0.4547 - acc: 0.7964 - val_loss: 0.4755 - val_acc: 0.7582
Epoch 2/10000
97693/97693 [==============================] - 5s - loss: 0.3947 - acc: 0.8352 - val_loss: 0.4790 - val_acc: 0.7915
Epoch 3/10000
97693/97693 [==============================] - 5s - loss: 0.3615 - acc: 0.8510 - val_loss: 0.4303 - val_acc: 0.8154
Epoch 4/10000
97693/97693 [==============================] - 5s - loss: 0.3401 - acc: 0.8605 - val_loss: 0.3097 - val_acc: 0.8584
Epoch 5/10000
97693/97693 [==============================] - 5s - loss: 0.3228 - acc: 0.8675 - val_loss: 0.3606 - val_acc: 0.8549
Epoch 6/10000
97693/97693 [==============================] - 5s - loss: 0.3112 - acc: 0.8731 - val_loss: 0.34

97659/97659 [==============================] - 5s - loss: 0.2905 - acc: 0.8821 - val_loss: 0.3470 - val_acc: 0.8452
Epoch 12/10000
97659/97659 [==============================] - 5s - loss: 0.2863 - acc: 0.8840 - val_loss: 0.3833 - val_acc: 0.8365
Epoch 13/10000
97659/97659 [==============================] - 5s - loss: 0.2828 - acc: 0.8852 - val_loss: 0.4903 - val_acc: 0.8093
Epoch 14/10000
97659/97659 [==============================] - 5s - loss: 0.2795 - acc: 0.8869 - val_loss: 0.3439 - val_acc: 0.8642
Epoch 15/10000
97659/97659 [==============================] - 5s - loss: 0.2769 - acc: 0.8876 - val_loss: 0.2882 - val_acc: 0.8740
Epoch 16/10000
97659/97659 [==============================] - 5s - loss: 0.2715 - acc: 0.8908 - val_loss: 0.8061 - val_acc: 0.7203
Epoch 17/10000
97659/97659 [==============================] - 5s - loss: 0.2718 - acc: 0.8908 - val_loss: 0.4032 - val_acc: 0.8445
Epoch 18/10000
97659/97659 [==============================] - 5s - loss: 0.2662 - acc: 0.8924 - va

97544/97544 [==============================] - 6s - loss: 0.2939 - acc: 0.8796 - val_loss: 0.4167 - val_acc: 0.8355
Epoch 9/10000
97544/97544 [==============================] - 5s - loss: 0.2870 - acc: 0.8835 - val_loss: 0.2954 - val_acc: 0.8522
Epoch 10/10000
97544/97544 [==============================] - 5s - loss: 0.2830 - acc: 0.8851 - val_loss: 0.3907 - val_acc: 0.8473
Epoch 11/10000
97544/97544 [==============================] - 6s - loss: 0.2772 - acc: 0.8871 - val_loss: 0.2669 - val_acc: 0.8737
Epoch 12/10000
97544/97544 [==============================] - 5s - loss: 0.2727 - acc: 0.8890 - val_loss: 0.3665 - val_acc: 0.8435
Epoch 13/10000
97544/97544 [==============================] - 5s - loss: 0.2684 - acc: 0.8918 - val_loss: 0.3118 - val_acc: 0.8593
Epoch 14/10000
97544/97544 [==============================] - 5s - loss: 0.2648 - acc: 0.8926 - val_loss: 0.3443 - val_acc: 0.8574
Epoch 15/10000
97544/97544 [==============================] - 5s - loss: 0.2626 - acc: 0.8943 - val

In [4]:
print(model_name)

metrics_kf

MLP


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.970911,0.757198,0.757198,0.558525,0.682628,0.571142,0.861502,00:04:56.887894
KF_2,0.967114,0.759036,0.759036,0.559256,0.707993,0.568739,0.845982,00:03:52.375400
KF_3,0.968360,0.765806,0.765806,0.554116,0.702728,0.558289,0.832807,00:04:08.726121
KF_4,0.974384,0.769231,0.769231,0.568099,0.712160,0.585242,0.866219,00:05:48.011362
KF_5,0.968261,0.766914,0.766914,0.553092,0.699132,0.556690,0.832018,00:04:13.079238
KF_6,0.968878,0.790803,0.790803,0.563131,0.724116,0.574108,0.846356,00:03:29.194491
KF_7,0.973293,0.772686,0.772686,0.564092,0.715620,0.577371,0.854512,00:05:08.958031
KF_8,0.972240,0.759889,0.759889,0.566486,0.682309,0.584724,0.879242,00:04:50.704065
KF_9,0.971913,0.774563,0.774563,0.569239,0.683820,0.588930,0.883831,00:06:37.341898
KF_10,0.966630,0.736981,0.740314,0.559593,0.665190,0.574771,0.875674,00:03:13.684673


In [ ]:
print('K-Fold: 10')

models.sort_values(['F1 score'], ascending=False)